In [9]:
#!pip install pickle
import pickle
from sklearn import datasets, svm
import numpy as np

#Carregando as features e target nas variaveis X e y
iris = datasets.load_iris()
X = iris.data
y = iris.target

#Caminho da onde eu salvarei o pkl
path = ""

#Treinando o modelo com SVM 
model = svm.SVC(kernel="poly", degree=3, C=1.0).fit(X,y)

#Salvando o Modelo no formato .pkl
pickle.dump(model, open(path+'modeloiris.pkl','wb' ))
print("Modelo Salvo com sucesso !")



Modelo Salvo com sucesso !


In [10]:
print(X)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [15]:
to_predict_list =[1,1,1,1]
to_predict_np_array = np.array(to_predict_list).reshape(1,4)
loaded_model = pickle.load(open('modeloiris.pkl', 'rb'))

result = loaded_model.predict(to_predict_np_array)
if result[0] == 0:
    prediction = "versicolor"
elif result[0] == 1:
    prediction = "virginica"
else:
    prediction = "setosa"

print(prediction)

versicolor


In [18]:
# skl2onnx
#onnxruntime
iris = datasets.load_iris()
X = iris.data
y = iris.target


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

RandomForestClassifier()

In [20]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
tensor_entrada = [('float_input', FloatTensorType([None,4]))]

modelo_onnx = convert_sklearn(clr, initial_types=tensor_entrada)
with open("modeloiris.onnx", "wb") as arquivo:
    arquivo.write(modelo_onnx.SerializeToString())

In [24]:
import onnxruntime as rt
sessao = rt.InferenceSession("modeloiris.onnx")
input_name = sessao.get_inputs()[0].name
label_name = sessao.get_outputs()[0].name

In [25]:
predicoes = sessao.run([label_name], {input_name: X_test.astype(np.float32)})[0]

In [26]:
print(predicoes)

[2 2 1 0 2 0 1 2 0 0 1 1 0 2 0 0 0 2 0 0 1 0 2 1 0 2 0 0 2 1 1 1 2 2 1 2 0
 0]


In [29]:
#Para o envio de predicts ao modelo, o array numpy deve conter duas dimensões [[ ]]
test_predicao = np.array([[1.0,1.0,1.0,1.0]], dtype = np.float32)
resultado_predicao_test = sessao.run([label_name], {input_name: test_predicao.astype(np.float32)})[0]

In [30]:
print(resultado_predicao_test)

[0]
